In [1]:
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

# Загрузим данные

In [28]:
# data = pd.DataFrame.from_csv("../train.csv")

In [ ]:
X = pd.DataFrame.from_csv('X_preprocessed')

In [27]:
test = pd.DataFrame.from_csv('test_preprocessed.csv')

In [ ]:
y = pd.DataFrame.from_csv('y.csv')

In [ ]:
# len(X[X['cancel_time']<0])*1./len(X)
# X = X.loc[X['cancel_time']<500]
# plt.figure(figsize=(16, 8))
# X_notminusone = X.loc[X['cancel_time']!=-1]
# trsh = plt.hist(np.log(np.array([1.]*len(X_notminusone)) + X_notminusone['cancel_time'].values), bins=2000)
# plt.xlim(0., 7.)
# plt.ylim(0., 3000.)

In [ ]:
# plt.figure(figsize=(16, 8))
# trash = plt.hist( data.loc[(data['burned']==1) & (data['cancel_time']<100000)]['cancel_time'], bins=1000 )
# plt.ylim(0, 5000)
# plt.xlim(0, 100000)

In [ ]:
# data.loc[(data['burned']==0)]['cancel_time'].value_counts()

In [ ]:
# data.loc[(data['burned']==1)]['cancel_time'].value_counts()

# предскажем cancel_time

### лол, cancel_time, не считая -1, имеет логнормальное распределение, значит его имеет смысл предсказывать

In [ ]:
cancel_times = data['cancel_time']

In [ ]:
print X.shape, cancel_times.shape

In [ ]:
cancel_times = np.log(cancel_times + np.array([2.]*len(cancel_times)))

In [ ]:
trash = plt.hist(cancel_times[cancel_times != 0], bins=100)

In [ ]:
params = {} 
params['objective']           = "reg:linear"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.1
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = cancel_times[:bound], cancel_times[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
# dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 300
esr = 10
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)

In [ ]:
gbm.best_ntree_limit

In [ ]:
dtrain = xgb.DMatrix(X, cancel_times)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 25)

In [ ]:
pred_cancel_time = gbm.predict(dtest, ntree_limit=25)

In [ ]:
pd.DataFrame(pred_cancel_time).to_csv('predicted_cancel_times.csv')

In [ ]:
dtrain_ = xgb.DMatrix(X)
train_pred_cancel_time = gbm.predict(dtrain_, ntree_limit=25)

In [ ]:
pd.DataFrame(train_pred_cancel_time).to_csv('train_predicted_cancel_times.csv')

In [ ]:
###
pred_cancel_times = pd.read_csv('predicted_cancel_times.csv')
train_pred_cancel_times = pd.read_csv('train_predicted_cancel_times.csv')

### ! Теперь у нас есть pred_cancel_time для test!!!!!

# предскажем driver_found

In [ ]:
driver_founds = data['driver_found']

In [ ]:
params = {} 
params['objective']           = "binary:logistic"
# params['booster']             = "gbtree"
# params['eval_metric']         = "auc"
params['eta']                 = 0.05
# params['max_depth']           = 7
# params['subsample']           = 0.681
# params['colsample_bytree']    = 0.95
params['seed']                = 1337
params['njobs']               = -1
# params['silent']              = 1
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = driver_founds[:bound], driver_founds[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
# dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 300
esr = 10
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)

In [ ]:
gbm.best_ntree_limit

In [ ]:
dtrain = xgb.DMatrix(X, driver_founds)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 28)

In [ ]:
pred_driver_founds = gbm.predict(dtest, ntree_limit=28)

In [ ]:
pd.DataFrame(pred_driver_founds).to_csv('predicted_driver_founds.csv')

In [ ]:
dtrain_ = xgb.DMatrix(X)
train_pred_driver_founds = gbm.predict(dtrain_, ntree_limit=25)

In [ ]:
pd.DataFrame(train_pred_driver_founds).to_csv('train_predicted_driver_founds.csv')

In [ ]:
###
pred_driver_founds = pd.read_csv('predicted_driver_founds.csv')
train_pred_driver_founds = pd.read_csv('train_predicted_driver_founds.csv')

# append cancels and drivers

### for train

In [ ]:
train_pred_cancel_times.index = X.index
del train_pred_cancel_times['Unnamed: 0']
X['cancel_times'] = train_pred_cancel_times
train_pred_driver_founds.index = X.index
del train_pred_driver_founds['Unnamed: 0']
X['driver_found'] = train_pred_driver_founds
X.head(1)

### for test

In [ ]:
pred_cancel_times.index = test.index
del pred_cancel_times['Unnamed: 0']
test['cancel_times'] = pred_cancel_times
pred_driver_founds.index = test.index
del pred_driver_founds['Unnamed: 0']
test['driver_found'] = pred_driver_founds
test.head(1)

In [ ]:
X.to_csv('X_with_metas.csv')
test.to_csv('test_with_metas.csv')

# если всё пошло по пизде

In [ ]:
X = pd.read_csv('X_with_metas.csv')
test = pd.read_csv('test_with_metas.csv')

# добавим vectorized кластеры

### в train

In [ ]:
# Xapp1 = X[:10].copy()
# Xapp2 = X[10:20].copy()
# Xapp2.index = Xapp1.index

# Xres = pd.concat([Xapp1, Xapp2[['lat','lon']]], axis=1,join_axes=[Xapp1.index])
# Xres

In [ ]:
X.head()

In [ ]:
trainclusters = pd.read_csv('train_clusters_vectorized.csv')

In [ ]:
del trainclusters['Unnamed: 0']
trainclusters.head()

In [ ]:
X = pd.concat([X, trainclusters], axis=1, join_axes=[X.index])

In [ ]:
X.head()

### в test

In [ ]:
test.head()

In [ ]:
testclusters.head()

In [ ]:
testclusters = pd.read_csv('test_clusters_vectorized.csv')

In [ ]:
del testclusters['Unnamed: 0']

In [ ]:
test = pd.concat([test, testclusters], axis=1, join_axes=[test.index])

In [ ]:
test.head()

# LEARN

In [ ]:
del X['Unnamed: 0']
del X['Unnamed: 0.1']
X.head()

In [ ]:
del test['Unnamed: 0']
del test['Unnamed: 0.1']
test.head()

In [ ]:
X.to_csv('X_clust_meta.csv')
test.to_csv('test_clust_meta.csv')

In [ ]:
X = pd.read_csv('X_clust_meta.csv')
test = pd.read_csv('test_clust_meta.csv')

# try ensemble


In [ ]:
bst = xgb.Booster({'nthread':16}) #init model
bst.load_model("clust_meta.model") # load data

In [ ]:
dtest_X = xgb.DMatrix(X)

In [ ]:
prediction_X = bst.predict(dtest_X, ntree_limit=2000)

In [ ]:
X['xgb2000'] = prediction_X

In [ ]:
dtest = xgb.DMatrix(test)

In [ ]:
prediction_test = bst.predict(dtest, ntree_limit=2000)

In [ ]:
test['xgb2000'] = prediction_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Также обучим более "случайную" версию RandomForest
model = RandomForestClassifier(n_estimators=1000,n_jobs=16,min_samples_split=75,min_samples_leaf=20)

model.fit(X, y)

rf1k_test = model.predict_proba(test)
rf1k_X = model.predict_proba(X)

In [ ]:
# sklearn.tree.DecisionTreeClassifier, RandomForestClassifier, AdaBoostClassifier, LinearModel.Ridge
# >>> from sklearn.ensemble import BaggingClassifier
# >>> from sklearn.neighbors import KNeighborsClassifier
# >>> bagging = BaggingClassifier(KNeighborsClassifier(),
# ...                             max_samples=0.5, max_features=0.5)
# class sklearn.ensemble.BaggingClassifier(base_estimator=None, n_estimators=10, 
#                                          max_samples=1.0, max_features=1.0, bootstrap=True, 
#                                          bootstrap_features=False, oob_score=False, 
#                                          warm_start=False, n_jobs=1, 
#                                          random_state=None, verbose=0)[source]
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(KNeighborsClassifier(), 
                            n_estimators = 200, 
                            max_samples=0.5, 
                            max_features=0.5,
                            n_jobs=16)


# final count

In [ ]:
del X['Unnamed: 0']
X.head()

In [ ]:
del test['Unnamed: 0']
test.head()

In [ ]:
# del y['Unnamed: 0']
y = pd.DataFrame(y)

In [ ]:
y.index = np.arange(0, len(y))

In [ ]:
y = pd.Series(y['burned'])

In [ ]:
params = {} 
params['objective']           = "binary:logistic" 
params['booster']             = "gbtree"
params['eval_metric']         = "auc"
params['eta']                 = 0.006
params['max_depth']           = 7
params['subsample']           = 0.681
params['colsample_bytree']    = 0.95
params['seed']                = 18
params['njobs']               = -1
params['silent']              = 1

In [ ]:
X.shape, y.shape, test.shape

In [ ]:
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = y[:bound], y[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
dtest = xgb.DMatrix(test)
watchlist = [(dval,'eval')]

In [ ]:
num_trees = 5000
esr = 12
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, verbose_eval=True, early_stopping_rounds = esr)
gbm.best_ntree_limit

In [ ]:
dtrain = xgb.DMatrix(X, y)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, 2000)
gbm.save_model('clust_meta.model')

In [ ]:
prediction = gbm.predict(dtest, ntree_limit=2000)

# save

In [ ]:
response = pd.DataFrame()
response["Ids"] = np.arange(test.shape[0])
response["Y_prob"] = prediction
 
response.to_csv('stacksub.csv',index=None)

In [ ]:
# prediction = gbm.predict(dtrain, ntree_limit=780)

In [ ]:
# response = pd.DataFrame()
# response["Ids"] = np.arange(X.shape[0])
# response["Y_prob"] = prediction
 
# response.to_csv('train779.csv',index=None)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Также обучим более "случайную" версию RandomForest
model = RandomForestClassifier(n_estimators=1000,n_jobs=16,min_samples_split=75,min_samples_leaf=20)

model.fit(X, y)

In [ ]:
rf1k_test = model.predict_proba(test)
rf1k_X = model.predict_proba(X)

In [ ]:
rf1k_X[:5]

In [ ]:
# res = res[:, 1]

response = pd.DataFrame()
response["Ids"] = np.arange(test.shape[0])
response["Y_prob"] = res
 
response.to_csv('rf_test779.csv',index=None)

In [ ]:
res = model.predict_proba(X)

# res = res[:, 1]

response = pd.DataFrame()
response["Ids"] = np.arange(X.shape[0])
response["Y_prob"] = res
 
response.to_csv('rf_train779.csv',index=None)

In [ ]:
np.corr(res, prediction)

In [ ]:
# from datanight import plot_forest_feature_importances
# plot_forest_feature_importances(model,X.columns)

In [ ]:
# from datanight import generate_submission
# generate_submission(preprocess_data,model,"submission.csv")

# Советы по улучшению модели:

* Смотри глазами на данные - строй графики, исследуй аномалия 
 * это может принести новые идеи и отбросить нежизнеспособные намного раньше 
 * Туториал по библиотеке, в которой можно строить графики 
   * http://matplotlib.org/users/pyplot_tutorial.html
 * Ключевой вопрос самому себе - __"От чего ещё может зависеть, сгорит ли заказ?"__


* Попробуй более точно настроить модель или выбрать другую
 * Random Forest с текущими параметрами можно улучшить
   * Документация по нему - http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
 * А можно использовать другие, более подходящие методы
   * Документация по методам - http://scikit-learn.org/stable/supervised_learning.html
 * Наконец, модели можно комбинировать
   * Например, усреднять с весами
   * В итоге качество может быть лучше, чем у каждой модели по отдельности
 * Важно не увлечься - зачастую качественные изменения решения намного лучше подкручивания параметров


* Подумай, какие ещё дополнительные данные можно провязать с выборкой? 
 * насколько реально применить их за отведённое время?


* __Главное__ - пытайся понять, дадут ли твои улучшения прирост на новых данных
 * Тестовая выборка находится по времени дальше, чем обучающая.
   * Например, нельзя использовать номер дня в году, потому что мы обучаемся на выборке __ДО__ начала контрольной
 * Это не значит, что нужно отправлять миллион решений на проверку и выбирать лучшее - это приведёт к переобучению
 